## Pandas
* numpy üzerine inşa edilmiştir.
* Çalışılan verinin 5-6 katı kadar memory olması beklenir.
* Daha büyük veri kümeleri için lazy çalışan Vaex kütüphanesi
    * Spark veya hadoop
* 2 tip mevcut
    1. Series
        * kolona benzetebiliriz. Aynı tipten veri tutar.
        * Sıra numarası ve etiket mevcut
    2. DataFrame
        * İki boyutludur. Tabloya benzetebiliriz.
        * Satır ve sütunların, sıra numarası ve etiketi mevcut.
* Broadcast özelliği her iki nesnede de mevcut

In [2]:
import numpy as np
import pandas as pd

In [3]:
#seri
pd.Series([11,22,33]) #satır etiketi ve sıra numarası bulunur.

pd.Series([11,22,33], dtype="str", index=["s1","s2","s3"]) 
#index satır etiketidir. satır etiketi vermezse sıra numarası etiket olur.

s1    11
s2    22
s3    33
dtype: object

In [4]:
#DataFrame
departman = pd.DataFrame(
                    data=[
                            (1,"Departman A"),
                            (2,"Departman B"),
                            (3,"Departman C")
                        ],
                        columns=["DepId","DepAd"]                        
                        )

#-
departman

# DepId kolonu satır etiketi olsun
departman.set_index("DepId", inplace=True) #orjinalini inplace=True ile etkiler.

#
departman.sort_index(ascending=False)
departman.sort_values(by="DepAd")


#indexi sıfırlamak
departman.reset_index()

,DepId,DepAd
0,1,Departman A
1,2,Departman B
2,3,Departman C


In [5]:
#DataFrame2
#dict keyleri kolon adı olur.
ekipEfor = pd.DataFrame(
                data= {
                        "Id":[1,2,3,4,5],
                        "Ad":["Ali","Veli","Ayşe","Fatma","Tonguç"],
                        "Cinsiyet":["E","E",None,"K","E"],
                        "Miktar":[120,None,250,350,None]
                        },
                columns=["Id","Ad","Cinsiyet","Miktar","DepId"], #yeni kolon eklenebilir
                index=["İstanbul","İstanbul","Ankara","Ankara","US"]
)

#--
ekipEfor

,Id,Ad,Cinsiyet,Miktar,DepId
İstanbul,1,Ali,E,120.0,NaN
İstanbul,2,Veli,E,NaN,NaN
Ankara,3,Ayşe,None,250.0,NaN
Ankara,4,Fatma,K,350.0,NaN
US,5,Tonguç,E,NaN,NaN


In [6]:
ekipEfor.rename(columns={"Miktar":"Uretim"}, index={"Ankara":"Bolu"}, inplace=True)

#---
ekipEfor.columns #kolonları görebilriiz.
ekipEfor.columns = ['Id', 'Ad', 'Cinsiyet', 'Uretim', 'DepId']

#
ekipEfor.index
ekipEfor.index.name="Lokasyon" #indexe ana başlık ekledik.

# Farklı teknikler bir arada kullanılabilir.
ekipEfor.index = np.where(ekipEfor.index == "İstanbul", "Çorum", ekipEfor.index)

#
ekipEfor

,Id,Ad,Cinsiyet,Uretim,DepId
Çorum,1,Ali,E,120.0,NaN
Çorum,2,Veli,E,NaN,NaN
Bolu,3,Ayşe,None,250.0,NaN
Bolu,4,Fatma,K,350.0,NaN
US,5,Tonguç,E,NaN,NaN


#### Hakkında bilgi almak

In [7]:
#ekip = ekipEfor #direkt atama yapıldığında her iki değişken de etkilenir.
ekip = ekipEfor

#memoryde aynı yeri gösteriyoru
print(id(ekip), id(ekipEfor))
print(ekip is ekipEfor)


ekip = ekipEfor.copy() #verinin kopyasını alır memoryde farklı yerleri gösterir.
print(ekip is ekipEfor)

2867889858400 2867889858400
True
False


In [8]:
ekip.shape
ekip.size
#len(ekip)

#--
ekip = ekip.astype({
                        "Id":np.int8,
                        "Uretim":np.float16,
                        "Ad":str
                    })
#
ekip.info() #240.0+ bytes >  175.0+ bytes bytes

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, Çorum to US
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Id        5 non-null      int8   
 1   Ad        5 non-null      object 
 2   Cinsiyet  4 non-null      object 
 3   Uretim    3 non-null      float16
 4   DepId     0 non-null      object 
dtypes: float16(1), int8(1), object(3)
memory usage: 175.0+ bytes


In [9]:
ekip.head(2) #varsayılan 5 satır
ekip.tail(2) #alttan kaç satır

,Id,Ad,Cinsiyet,Uretim,DepId
Bolu,4,Fatma,K,350.0,NaN
US,5,Tonguç,E,NaN,NaN


In [10]:
ekip.describe() #sayısal alanlarla ilgili istaitstik
ekip.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,5.0,3.0,1.581139,1.0,2.0,3.0,4.0,5.0
Uretim,3.0,240.0,115.312500,120.0,185.0,250.0,300.0,350.0


### Bazı kullanışlı foksiyonlar

In [15]:
#axis verilebilir
ekip.sum() #varsayılan axis=0
ekip.sum(axis=1) 

#
ekip.count() #null olmayanlar

#min,max, std, mean, median, cumsum verilebilir.

Id          5
Ad          5
Cinsiyet    4
Uretim      3
DepId       0
dtype: int64

In [16]:
ekip.nunique() #kaç tane benzersiz kayıt var
#ekip.nunique(axis=1)

Id          5
Ad          5
Cinsiyet    2
Uretim      3
DepId       0
dtype: int64

In [17]:
#seri
ekip.Uretim.mean() #ortalama üretimi verir.

#
ekip.Cinsiyet.value_counts() #hangi değerden kaç tane var.

#count(distinct ) kaç farklı değer var.
ekip.Cinsiyet.value_counts().count()

2

### Null (None, NaN, N/A) Değerler

* `isna` veya `isnull` hücreleri kontrol eder. None olanlar True gelir.
* `notna` ve `notnull` isna nın tersi
* `all()` hepsi True mu?, `any()` hiç True var mı?
* `fillna` ile null alanlara değer atarız.
* `dropna` ile nulları kaldırabiliriz.

Not: fillna ve dropna axis alır. Yapılan işlem orjinalini etkilemez.

In [18]:
ekip.isna() #isnull()

#
#axis verilebilir
ekip.isna().any() #hiç True var mı? yani null var mı?
ekip.isna().all() #tamamı null mı?

#tabloda hiç null var mı?
ekip.isna().any().any()

True

In [19]:
ekip.Uretim.isna().sum() #üretim kolonunda kaç null var?

#hangileri null
ekip.Uretim.isna()
ekip.Uretim.isna().tolist()

#
ekip[ekip.Uretim.isna()] #tüm kolonlar gelecekse
ekip.loc[ekip.Uretim.isna(), ["Ad","Cinsiyet","Uretim"]] #etiketler kullanılackasa
ekip.iloc[ekip.Uretim.isna().tolist(), 1:4] #tolist ileliste yapmak gerekiyor. sıra numarası kullanılacaksa

#etiket üzerinden slicer yapılabilir.
ekip.loc[ekip.Uretim.isna(), "Ad":"Uretim"]

,Ad,Cinsiyet,Uretim
Çorum,Veli,E,NaN
US,Tonguç,E,NaN


In [94]:
#Soru:
#fillna kullanmadan NULL olan üretimlere ortalama üretimi basalım.
ekip.loc[ekip.Uretim.isna(), "Uretim"] = ekip.Uretim.mean()
#ekip[ekip.Uretim.isna()].Uretim
ekip

,Id,Ad,Cinsiyet,Uretim,DepId
Çorum,1,Ali,E,120.0,NaN
Çorum,2,Veli,E,240.0,NaN
Bolu,3,Ayşe,None,250.0,NaN
Bolu,4,Fatma,K,350.0,NaN
US,5,Tonguç,E,240.0,NaN


In [105]:
#fillna
ekip = ekipEfor.copy()

#
#ekip.fillna("Bilinmiyor") #tüm null hücrelere değer verir.

#
ekip.fillna({
                "Uretim":ekip.Uretim.mean(),
                #"Cinsiyet":"Bilinmiyor",
                "Cinsiyet":ekip.Cinsiyet.mode()[0],
                "DepId":-1
})

,Id,Ad,Cinsiyet,Uretim,DepId
Çorum,1,Ali,E,120.0,-1
Çorum,2,Veli,E,240.0,-1
Bolu,3,Ayşe,E,250.0,-1
Bolu,4,Fatma,K,350.0,-1
US,5,Tonguç,E,240.0,-1


In [108]:
#fillna
ekip = ekipEfor.copy()

#
#ekip.dropna(axis=1, how="any") #herhangi bir null varsa axisi kaldır
ekip.dropna(axis=1, how="all") #hepsi null ise kaldır

#
ekip.dropna(axis=0, how="all") #boş satırı komple kaldır

,Id,Ad,Cinsiyet,Uretim,DepId
Çorum,1,Ali,E,120.0,NaN
Çorum,2,Veli,E,NaN,NaN
Bolu,3,Ayşe,None,250.0,NaN
Bolu,4,Fatma,K,350.0,NaN
US,5,Tonguç,E,NaN,NaN


#### Kolan Ekleme ve kaldırma

In [21]:
hedef = 500
ekip["HedefDurumu"] = ekip.Uretim / hedef 

In [22]:
ekip

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu
Çorum,1,Ali,E,120.0,NaN,0.24
Çorum,2,Veli,E,NaN,NaN,NaN
Bolu,3,Ayşe,None,250.0,NaN,0.50
Bolu,4,Fatma,K,350.0,NaN,0.70
US,5,Tonguç,E,NaN,NaN,NaN


In [24]:
# eval expression yardımıyla kolon üretmemizi sağlayacak

buyumeOrani = 1.2

# @ ile dışarıdaki değişkeni çağırabiliriz.
ekip.eval("""
            GelecekYilUretim = Uretim * @buyumeOrani
            SonrakiYilUretim = GelecekYilUretim * @buyumeOrani
            """, inplace=True)
ekip

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Çorum,1,Ali,E,120.0,NaN,0.24,144.0,172.875
Çorum,2,Veli,E,NaN,NaN,NaN,NaN,NaN
Bolu,3,Ayşe,None,250.0,NaN,0.50,300.0,360.000
Bolu,4,Fatma,K,350.0,NaN,0.70,420.0,504.000
US,5,Tonguç,E,NaN,NaN,NaN,NaN,NaN


In [27]:
ekip.drop(['HedefDurumu', 'GelecekYilUretim'], axis=1) # kolon

ekip.drop(index='Çorum') # satır

ekip.drop(index='Bolu', columns='Ad') # satır ve kolon etiketi

,Id,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Çorum,1,E,120.0,NaN,0.24,144.0,172.875
Çorum,2,E,NaN,NaN,NaN,NaN,NaN
US,5,E,NaN,NaN,NaN,NaN,NaN


In [51]:
# soru:
"""
    1. Cinsiyet alanı ve üretim alanı null olmayanlar
    2. Cinsiyet alanı null olanlar
    3. Uretim alanı null olanlar
    4. Uretim alanı null olmayanlar
    5. Üretimi null olmayan erkek üyeler
    6. Ortalama üretimin üzerinde olan kadın üyeler
"""

'\n    1. Cinsiyet alanı ve üretim alanı null olmayanlar\n    2. Cinsiyet alanı null olanlar\n    3. Uretim alanı null olanlar\n    4. Uretim alanı null olmayanlar\n    5. Üretimi null olmayan erkek üyeler\n    6. Ortalama üretimin üzerinde olan kadın üyeler\n'

In [41]:
ekip[ekip['Cinsiyet'].notnull() & ekip['Uretim'].notnull()]

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Çorum,1,Ali,E,120.0,NaN,0.24,144.0,172.875
Bolu,4,Fatma,K,350.0,NaN,0.70,420.0,504.000


In [43]:
ekip.query("Cinsiyet.isna()")

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Bolu,3,Ayşe,None,250.0,NaN,0.5,300.0,360.0


In [45]:
#1.

ekip.query("Cinsiyet == Cinsiyet & Uretim == Uretim") # None ile None eşit mi sorusu False üretir.

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Çorum,1,Ali,E,120.0,NaN,0.24,144.0,172.875
Bolu,4,Fatma,K,350.0,NaN,0.70,420.0,504.000


In [46]:
#2. 
ekip.query("Cinsiyet != Cinsiyet")

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Bolu,3,Ayşe,None,250.0,NaN,0.5,300.0,360.0


In [47]:
#3.

ekip.query("Cinsiyet.isnull()")

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Bolu,3,Ayşe,None,250.0,NaN,0.5,300.0,360.0


In [48]:
#4.

ekip.query("Uretim.notna()")

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Çorum,1,Ali,E,120.0,NaN,0.24,144.0,172.875
Bolu,3,Ayşe,None,250.0,NaN,0.50,300.0,360.000
Bolu,4,Fatma,K,350.0,NaN,0.70,420.0,504.000


In [49]:
#5.

ekip.query("Uretim.notna() and Cinsiyet == 'E'")

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Çorum,1,Ali,E,120.0,NaN,0.24,144.0,172.875


In [52]:
#6.

ekip.query(f"Uretim > {ekip.Uretim.mean()}")

ortalama_uretim = ekip.Uretim.mean()
ekip.query("Uretim > @ortalama_uretim")

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Bolu,3,Ayşe,None,250.0,NaN,0.5,300.0,360.0
Bolu,4,Fatma,K,350.0,NaN,0.7,420.0,504.0


### isin
* tüm hücrelerde belli bir değeri aramıızı sağlar.
* Geriye hücrelerde True/False döner

In [55]:
ekip.isin(["E", "Ali"])
ekip.isin(["E", "Ali"]).sum()

Id                  0
Ad                  1
Cinsiyet            3
Uretim              0
DepId               0
HedefDurumu         0
GelecekYilUretim    0
SonrakiYilUretim    0
dtype: int64

In [57]:
ekip.isin({"Ad":['Ali', 'Ayşe'], "Cinsiyet": ['E']}) # sonuçlar filtre olarak kullanılabilir.

,Id,Ad,Cinsiyet,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Çorum,False,True,True,False,False,False,False,False
Çorum,False,False,True,False,False,False,False,False
Bolu,False,True,False,False,False,False,False,False
Bolu,False,False,False,False,False,False,False,False
US,False,False,True,False,False,False,False,False


#### filter
* etiketleri tarif etmemizi sağlıyor.

In [58]:
#soru:
#içinde im geçen kolonlar gelsin.

In [64]:
ekip.columns

ekip.columns.str.contains("im")

ekip.loc[:, ekip.columns.str.contains("im")]

,Uretim,GelecekYilUretim,SonrakiYilUretim
Çorum,120.0,144.0,172.875
Çorum,NaN,NaN,NaN
Bolu,250.0,300.0,360.000
Bolu,350.0,420.0,504.000
US,NaN,NaN,NaN


In [69]:
ekip.filter(items=['Ad','Uretim'])

ekip.filter(like='e')

ekip.filter(like='o', axis=0)

ekip.filter(regex='im$')

ekip.filter(regex='^I')

,Id
Çorum,1
Çorum,2
Bolu,3
Bolu,4
US,5


### Groupby ve Agg

In [70]:
ekip.count()

Id                  5
Ad                  5
Cinsiyet            4
Uretim              3
DepId               0
HedefDurumu         3
GelecekYilUretim    3
SonrakiYilUretim    3
dtype: int64

In [71]:
ekip.groupby(by='Cinsiyet').count()

,Id,Ad,Uretim,DepId,HedefDurumu,GelecekYilUretim,SonrakiYilUretim
Cinsiyet,,,,,,,
E,3,3,1,0,1,1,1
K,1,1,1,0,1,1,1


In [73]:
ekip.groupby(by='Cinsiyet').agg(["sum","mean","count"])

Id                 Uretim              HedefDurumu              \
         sum      mean count    sum   mean count         sum  mean count   
Cinsiyet                                                                   
E          8  2.666667     3  120.0  120.0     1        0.24  0.24     1   
K          4  4.000000     1  350.0  350.0     1        0.70  0.70     1   

         GelecekYilUretim              SonrakiYilUretim                 
                      sum   mean count              sum     mean count  
Cinsiyet                                                                
E                   144.0  144.0     1          172.875  172.875     1  
K                   420.0  420.0     1          504.000  504.000     1

In [75]:
df = ekip.groupby(by='Cinsiyet').agg({"Uretim":["min","max","sum","mean"], "Id":["min","max"]})
df

Uretim                       Id    
            min    max    sum   mean min max
Cinsiyet                                    
E         120.0  120.0  120.0  120.0   1   5
K         350.0  350.0  350.0  350.0   4   4

In [77]:
# hiyerarşik indexlerdeki bir alana erişmek
df = df.reset_index()
df

Cinsiyet Uretim                       Id    
              min    max    sum   mean min max
0        E  120.0  120.0  120.0  120.0   1   5
1        K  350.0  350.0  350.0  350.0   4   4

In [84]:
df.loc[:,"Uretim"]
df.loc[:,("Uretim", ["min","max"])]

Uretim       
     min    max
0  120.0  120.0
1  350.0  350.0